## Financial Services

In [32]:
!pip install -r requirements.txt --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.document_loaders import UnstructuredURLLoader
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain_community.vectorstores import Qdrant
from langchain.chains import RetrievalQA
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams
from langchain_huggingface import HuggingFaceEmbeddings
from cleantext import clean

In [34]:
!pip install python-dotenv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [35]:
from dotenv import load_dotenv
import os
import nltk
import time

In [37]:
# Load environment variables from .env file
load_dotenv()

# Access the environment variables
qdrant_api_key = "7hCtBhv8qcjRZ6O6DdeEIvd_lXkTombKa64-MrO6nsdLptTWecPFtQ"


In [38]:
print(qdrant_api_key)

7hCtBhv8qcjRZ6O6DdeEIvd_lXkTombKa64-MrO6nsdLptTWecPFtQ


#### Wikipedia

In [7]:
from langchain_community.document_loaders import WikipediaLoader

In [39]:
docs = WikipediaLoader(query="International Financial Services Centres Authority", doc_content_chars_max = 100000, load_max_docs=1).load()

In [40]:
docs[0].metadata  # meta-information of the Document

{'title': 'International Financial Services Centres Authority',
 'summary': 'The International Financial Services Centres Authority (IFSCA) is the regulatory body for the Indian special economic zones such as the GIFT International Financial Services Centre for International Financial Services and commodity markets under the ownership of the Government of India. It was established in 2020, under the International Financial Services Centres Authority Act, 2019. The International Financial Services Centre (IFSC) is located in Gujarat International Finance Tec-City (GIFT City).',
 'source': 'https://en.wikipedia.org/wiki/International_Financial_Services_Centres_Authority'}

In [41]:
print(docs[0].page_content)

The International Financial Services Centres Authority (IFSCA) is the regulatory body for the Indian special economic zones such as the GIFT International Financial Services Centre for International Financial Services and commodity markets under the ownership of the Government of India. It was established in 2020, under the International Financial Services Centres Authority Act, 2019. The International Financial Services Centre (IFSC) is located in Gujarat International Finance Tec-City (GIFT City).


== History ==


=== Developments that led to the IFSCA ===
In December 2015 the first International Financial Services Centre (IFSC) in India was set up in Gujarat International Finance Tec-City (GIFT City).
In 2019 the Government of India enacted an Act of Parliament called the International Financial Services Centres Authority Act, 2019. 


=== Establishment ===
On April 27, 2020, The IFSCA was established as a statutory body under the International Financial Services Centres Authority 

In [42]:
documents = []
documents.extend(docs)

In [43]:
len(documents)

1

In [44]:
len(documents[0].page_content)

9374

In [45]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
texts = text_splitter.split_documents(documents)

In [46]:
len(texts)

27

In [47]:
model_name = 'sentence-transformers/all-MiniLM-L12-v1'

In [48]:
# Define model parameters
model_kwargs = {'device': 'cpu', 'trust_remote_code': True}
encode_kwargs = {'normalize_embeddings': True}

# Initialize HuggingFace embeddings
embeddings_model = HuggingFaceEmbeddings(model_name=model_name,model_kwargs=model_kwargs,encode_kwargs=encode_kwargs)

In [49]:
# Define the URL for Qdrant
url="https://a1fbbfce-ca47-437d-b308-e79724650692.us-east4-0.gcp.cloud.qdrant.io:6333"

In [50]:
# Initialize Qdrant client
client =  QdrantClient(url=url, api_key=qdrant_api_key)

In [51]:
# Get vector size by embedding a test query
vector_size = len(embeddings_model.embed_query("test"))
# Collection name
collection_name = "finance_collection"

In [52]:
# Check if the collection already exists
if client.collection_exists(collection_name):
    # Create a collection with the calculated vector size
    client.delete_collection(collection_name=collection_name)
    client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=vector_size, distance="Dot")
        )
else:
    client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=vector_size, distance="Dot")
        )

In [53]:
start =time.time()
# Insert documents into the collection
qdrant = Qdrant.from_documents(texts,embeddings_model,url=url,prefer_grpc=False,api_key=qdrant_api_key,collection_name=collection_name,force_recreate=False,distance_func="Dot")
end =time.time()
print("Time taken is:", end-start,"seconds")

Time taken is: 1.0052828788757324 seconds


In [54]:
query = "What's the organisation structure?"
qdrant.similarity_search_with_score(query)

[(Document(metadata={'source': 'https://en.wikipedia.org/wiki/International_Financial_Services_Centres_Authority', 'summary': 'The International Financial Services Centres Authority (IFSCA) is the regulatory body for the Indian special economic zones such as the GIFT International Financial Services Centre for International Financial Services and commodity markets under the ownership of the Government of India. It was established in 2020, under the International Financial Services Centres Authority Act, 2019. The International Financial Services Centre (IFSC) is located in Gujarat International Finance Tec-City (GIFT City).', 'title': 'International Financial Services Centres Authority', '_id': '5632b19a-61f5-40d8-ac04-75495190d274', '_collection_name': 'finance_collection'}, page_content='== Organisation structure ==\nThe International Financial Services Centres Authority (IFSCA) is headed by a chairperson who is appointed by the Government of India. The chairperson is assisted by a f

## LLM result

In [ ]:
# from langchain_openai import ChatOpenAI
# from langchain.chains import RetrievalQA
# # completion llm
# llm = ChatOpenAI(
#     openai_api_key=os.getenv("OPENAI_API_KEY"),
#     model_name='gpt-4o-mini',
#     temperature=0.0
# )
# qa = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=qdrant.as_retriever()
# )
# qa.run(query)

#### URL

In [55]:
#n_weblinks = int(input("How many web links you want the RAG agent to refer for response generation & insights? Enter here: "))
urls = []
inp = input("Enter Link:")
urls.append(inp)
print("URL:", urls)

Enter Link: https://finance.ec.europa.eu/financial-crime/anti-money-laundering-and-countering-financing-terrorism-eu-level_en#legislation


URL: ['https://finance.ec.europa.eu/financial-crime/anti-money-laundering-and-countering-financing-terrorism-eu-level_en#legislation']


In [ ]:
#https://finance.ec.europa.eu/financial-crime/anti-money-laundering-and-countering-financing-terrorism-eu-level_en#legislation

In [56]:
loader = UnstructuredURLLoader(urls=urls, show_progress_bar=True)
urls = loader.load()

[nltk_data] Downloading package punkt to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
100%|██████████| 1/1 [00:00<00:00,  2.08it/s]


In [60]:
urls[0].metadata

{'source': 'https://finance.ec.europa.eu/financial-crime/anti-money-laundering-and-countering-financing-terrorism-eu-level_en#legislation'}

In [63]:
raw_text =urls[0].page_content
len(raw_text)

17220

In [64]:
clean_url_text = clean(text=raw_text,
            fix_unicode=True,
            to_ascii=True,
            lower=True,
            no_line_breaks=False,
            no_urls=False,
            no_emails=False,
            no_phone_numbers=False,
            no_numbers=False,
            no_digits=False,
            no_currency_symbols=False,
            no_punct=False,
            lang="en"
            )

In [65]:
len(clean_url_text)

17037

In [67]:
from langchain.schema import Document
metadata = {
    "source": 'https://finance.ec.europa.eu/financial-crime/anti-money-laundering-and-countering-financing-terrorism-eu-level_en#legislation'
}

doc = Document(page_content=clean_url_text, metadata=metadata)
document =[doc]

In [68]:
document

[Document(metadata={'source': 'https://finance.ec.europa.eu/financial-crime/anti-money-laundering-and-countering-financing-terrorism-eu-level_en#legislation'}, page_content="page contents\nwhat the eu is doing and why\nit is essential that gatekeepers (banks and other obliged entities) apply measures to prevent money laundering and terrorist financing. traceability of financial information has an important deterrent effect.\nthe european commission carries out risk assessments in order to identify and respond to risks affecting the euinternal market.the european union adopted robust legislation to fight against money laundering and terrorist financing (aml). the first anti-money laundering directive was adopted in 1990 in order to prevent the misuse of the financial system for the purpose of money laundering. it provides that obliged entities shall apply customer due diligence requirements when entering into a business relationship (i.e. identify and verify the identity of clients, mon

In [69]:
print(documents[0].page_content)

The International Financial Services Centres Authority (IFSCA) is the regulatory body for the Indian special economic zones such as the GIFT International Financial Services Centre for International Financial Services and commodity markets under the ownership of the Government of India. It was established in 2020, under the International Financial Services Centres Authority Act, 2019. The International Financial Services Centre (IFSC) is located in Gujarat International Finance Tec-City (GIFT City).


== History ==


=== Developments that led to the IFSCA ===
In December 2015 the first International Financial Services Centre (IFSC) in India was set up in Gujarat International Finance Tec-City (GIFT City).
In 2019 the Government of India enacted an Act of Parliament called the International Financial Services Centres Authority Act, 2019. 


=== Establishment ===
On April 27, 2020, The IFSCA was established as a statutory body under the International Financial Services Centres Authority 

In [70]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
texts = text_splitter.split_documents(document)

In [71]:
len(texts)

45

In [72]:
collection_name = "finance_collection"

In [73]:
start =time.time()
# Insert documents into the collection
qdrant = Qdrant.from_documents(texts,embeddings_model,url=url,prefer_grpc=False,api_key=qdrant_api_key,collection_name=collection_name,force_recreate=False,distance_func="Dot")
end =time.time()
print("Time taken is:", end-start,"seconds")

Time taken is: 1.7736268043518066 seconds


In [75]:
query = "What's the organisation structure?"
qdrant.similarity_search_with_score(query)

[(Document(metadata={'source': 'https://en.wikipedia.org/wiki/International_Financial_Services_Centres_Authority', 'summary': 'The International Financial Services Centres Authority (IFSCA) is the regulatory body for the Indian special economic zones such as the GIFT International Financial Services Centre for International Financial Services and commodity markets under the ownership of the Government of India. It was established in 2020, under the International Financial Services Centres Authority Act, 2019. The International Financial Services Centre (IFSC) is located in Gujarat International Finance Tec-City (GIFT City).', 'title': 'International Financial Services Centres Authority', '_id': '5632b19a-61f5-40d8-ac04-75495190d274', '_collection_name': 'finance_collection'}, page_content='== Organisation structure ==\nThe International Financial Services Centres Authority (IFSCA) is headed by a chairperson who is appointed by the Government of India. The chairperson is assisted by a f

In [ ]:
# from langchain_openai import ChatOpenAI
# from langchain.chains import RetrievalQA
# # completion llm
# llm = ChatOpenAI(
#     openai_api_key=os.getenv("OPENAI_API_KEY"),
#     model_name='gpt-3.5-turbo',
#     temperature=0.0
# )
# qa = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=vectorstore.as_retriever()
# )
# print(qa.run(query))

## Healthcare & Data Privacy

#### Wikipedia

In [76]:
from langchain_community.document_loaders import WikipediaLoader

In [77]:
docs = WikipediaLoader(query="General Data Protection Regulation", doc_content_chars_max = 100000, load_max_docs=1).load()

In [83]:
docs[0].metadata  # meta-information of the Document

{'title': 'General Data Protection Regulation',
 'summary': 'The General Data Protection Regulation (Regulation (EU) 2016/679, abbreviated GDPR) is a European Union regulation on information privacy in the European Union (EU) and the European Economic Area (EEA). The GDPR is an important component of EU privacy law and human rights law, in particular Article 8(1) of the Charter of Fundamental Rights of the European Union. It also governs the transfer of personal data outside the EU and EEA. The GDPR\'s goals are to enhance individuals\' control and rights over their personal information and to simplify the regulations for international business.  It supersedes the Data Protection Directive 95/46/EC and, among other things, simplifies the terminology.\nThe European Parliament and Council of the European Union adopted the GDPR on 14 April 2016, to become effective on 25 May 2018. As an EU regulation (instead of a directive), GDPR is directly applicable with force of law on its own withou

In [85]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
texts = text_splitter.split_documents(documents)

In [86]:
# Collection name
collection_name = "DataProtection_collection"

In [87]:
# Check if the collection already exists
if client.collection_exists(collection_name):
    # Create a collection with the calculated vector size
    client.delete_collection(collection_name=collection_name)
    client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=vector_size, distance="Dot")
        )
else:
    client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=vector_size, distance="Dot")
        )

In [88]:
start =time.time()
# Insert documents into the collection
qdrant = Qdrant.from_documents(texts,embeddings_model,url=url,prefer_grpc=False,api_key=qdrant_api_key,collection_name=collection_name,force_recreate=False,distance_func="Dot")
end =time.time()
print("Time taken is:", end-start,"seconds")

Time taken is: 4.185298919677734 seconds


In [89]:
query = "What is GDPR?"
qdrant.similarity_search_with_score(query)

[(Document(metadata={'source': 'https://en.wikipedia.org/wiki/General_Data_Protection_Regulation', 'summary': 'The General Data Protection Regulation (Regulation (EU) 2016/679, abbreviated GDPR) is a European Union regulation on information privacy in the European Union (EU) and the European Economic Area (EEA). The GDPR is an important component of EU privacy law and human rights law, in particular Article 8(1) of the Charter of Fundamental Rights of the European Union. It also governs the transfer of personal data outside the EU and EEA. The GDPR\'s goals are to enhance individuals\' control and rights over their personal information and to simplify the regulations for international business.  It supersedes the Data Protection Directive 95/46/EC and, among other things, simplifies the terminology.\nThe European Parliament and Council of the European Union adopted the GDPR on 14 April 2016, to become effective on 25 May 2018. As an EU regulation (instead of a directive), GDPR is direc

In [91]:
# from langchain_openai import ChatOpenAI
# from langchain.chains import RetrievalQA
# # completion llm
# llm = ChatOpenAI(
#     openai_api_key=os.getenv("OPENAI_API_KEY"),
#     model_name='gpt-4o-mini',
#     temperature=0.0
# )
# qa = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=vectorstore.as_retriever()
# )
# qa.run(query)

#### URLs

In [90]:
#n_weblinks = int(input("How many web links you want the RAG agent to refer for response generation & insights? Enter here: "))
urls = []
inp = input("Enter Link:")
urls.append(inp)
print("URL:", urls)

Enter Link: https://www.proofpoint.com/us/threat-reference/ccpa-compliance#:~:text=CCPA%20compliance%20is%20a%20set%20of%20regulations%20that,implement%20reasonable%20security%20measures%20to%20protect%20user%20data


URL: ['https://www.proofpoint.com/us/threat-reference/ccpa-compliance#:~:text=CCPA%20compliance%20is%20a%20set%20of%20regulations%20that,implement%20reasonable%20security%20measures%20to%20protect%20user%20data']


In [91]:
loader = UnstructuredURLLoader(urls=urls, show_progress_bar=True)
urls = loader.load()

100%|██████████| 1/1 [00:00<00:00,  2.35it/s]


In [92]:
raw_text =urls[0].page_content
len(raw_text)

14402

In [93]:
clean_url_text = clean(text=raw_text,
            fix_unicode=True,
            to_ascii=True,
            lower=True,
            no_line_breaks=False,
            no_urls=False,
            no_emails=False,
            no_phone_numbers=False,
            no_numbers=False,
            no_digits=False,
            no_currency_symbols=False,
            no_punct=False,
            lang="en"
            )

In [94]:
len(clean_url_text)

14270

In [95]:
from langchain.schema import Document
metadata = {
    "source": 'https://finance.ec.europa.eu/financial-crime/anti-money-laundering-and-countering-financing-terrorism-eu-level_en#legislation'
}

doc = Document(page_content=clean_url_text, metadata=metadata)
document =[doc]

In [97]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
texts = text_splitter.split_documents(document)
#texts

In [98]:
len(texts)

37

In [99]:
start =time.time()
# Insert documents into the collection
qdrant = Qdrant.from_documents(texts,embeddings_model,url=url,prefer_grpc=False,api_key=qdrant_api_key,collection_name=collection_name,force_recreate=False,distance_func="Dot")
end =time.time()
print("Time taken is:", end-start,"seconds")

Time taken is: 1.1621510982513428 seconds


In [100]:
query = "What is CCPA?"
qdrant.similarity_search_with_score(query)

[(Document(metadata={'source': 'https://finance.ec.europa.eu/financial-crime/anti-money-laundering-and-countering-financing-terrorism-eu-level_en#legislation', '_id': '5ef42ab5-24a9-4cfb-abfe-6780fd2026c8', '_collection_name': 'DataProtection_collection'}, page_content='definition\nthe california consumer privacy act (ccpa) was enacted in 2018 to combat the numerous data breaches in big tech from poorly defined access controls and privacy management. modeled after the european union (eu) general data protection regulation (gdpr), the new regulations give users the right to know when and how their information is being collected and sold, as well as the ability to opt-out.'),
  0.72756207),
 (Document(metadata={'source': 'https://finance.ec.europa.eu/financial-crime/anti-money-laundering-and-countering-financing-terrorism-eu-level_en#legislation', '_id': '2284e8a1-d00e-4d33-b7cf-95f1e7f29f20', '_collection_name': 'DataProtection_collection'}, page_content="ccpa compliance is a set of reg

In [ ]:
# from langchain_openai import ChatOpenAI
# from langchain.chains import RetrievalQA
# # completion llm
# llm = ChatOpenAI(
#     openai_api_key=os.getenv("OPENAI_API_KEY"),
#     model_name='gpt-4o-mini',
#     temperature=0.0
# )
# qa = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=vectorstore.as_retriever()
# )
# print(qa.run(query))

HEALTHCARE

**WIKIPEDIA**

In [101]:
#Food_and_Drug_Administration
docs = WikipediaLoader(query="Food_and_Drug_Administration", doc_content_chars_max = 100000, load_max_docs=1).load()


In [102]:
docs[0].metadata  # meta-information of the Document

{'title': 'Food and Drug Administration',
 'summary': "The United States Food and Drug Administration (FDA or US FDA) is a federal agency of the Department of Health and Human Services. The FDA is responsible for protecting and promoting public health through the control and supervision of food safety, tobacco products, caffeine products, dietary supplements, prescription and over-the-counter pharmaceutical drugs (medications), vaccines, biopharmaceuticals, blood transfusions, medical devices, electromagnetic radiation emitting devices (ERED), cosmetics, animal foods & feed and veterinary products.\nThe FDA's primary focus is enforcement of the Federal Food, Drug, and Cosmetic Act (FD&C). However, the agency also enforces other laws, notably Section 361 of the Public Health Service Act as well as associated regulations. Much of this regulatory-enforcement work is not directly related to food or drugs but involves other factors like regulating lasers, cellular phones, and condoms. In ad

In [103]:
documents = []
documents.extend(docs)

In [104]:
len(documents)

1

In [105]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
texts = text_splitter.split_documents(documents)

In [106]:
len(texts)

165

In [107]:
collection_name = "Healthcare_collection"

In [108]:
# Check if the collection already exists
if client.collection_exists(collection_name):
    # Create a collection with the calculated vector size
    client.delete_collection(collection_name=collection_name)
    client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=vector_size, distance="Dot")
        )
else:
    client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=vector_size, distance="Dot")
        )

In [109]:
start =time.time()
# Insert documents into the collection
qdrant = Qdrant.from_documents(texts,embeddings_model,url=url,prefer_grpc=False,api_key=qdrant_api_key,collection_name=collection_name,force_recreate=False,distance_func="Dot")
end =time.time()
print("Time taken is:", end-start,"seconds")

Time taken is: 4.154428243637085 seconds


In [110]:
query = "Summarize Post-market safety surveillance?"
qdrant.similarity_search(query)

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Food_and_Drug_Administration', 'summary': "The United States Food and Drug Administration (FDA or US FDA) is a federal agency of the Department of Health and Human Services. The FDA is responsible for protecting and promoting public health through the control and supervision of food safety, tobacco products, caffeine products, dietary supplements, prescription and over-the-counter pharmaceutical drugs (medications), vaccines, biopharmaceuticals, blood transfusions, medical devices, electromagnetic radiation emitting devices (ERED), cosmetics, animal foods & feed and veterinary products.\nThe FDA's primary focus is enforcement of the Federal Food, Drug, and Cosmetic Act (FD&C). However, the agency also enforces other laws, notably Section 361 of the Public Health Service Act as well as associated regulations. Much of this regulatory-enforcement work is not directly related to food or drugs but involves other factors like regul

**URLS**

In [111]:
#n_weblinks = int(input("How many web links you want the RAG agent to refer for response generation & insights? Enter here: "))
urls = []
inp = input("Enter Link:")
urls.append(inp)
print("URL:", urls)

Enter Link: https://advisera.com/13485academy/what-is-eu-mdr/#:~:text=Under%20the%20EU%20MDR%2C%20medical%20device%20manufacturers%20are,ensure%20ongoing%20compliance%20and%20safety%20of%20the%20devices


URL: ['https://advisera.com/13485academy/what-is-eu-mdr/#:~:text=Under%20the%20EU%20MDR%2C%20medical%20device%20manufacturers%20are,ensure%20ongoing%20compliance%20and%20safety%20of%20the%20devices']


In [112]:
loader = UnstructuredURLLoader(urls=urls, show_progress_bar=True)
urls = loader.load()

100%|██████████| 1/1 [00:00<00:00,  7.53it/s]


In [116]:
urls

[Document(metadata={'source': 'https://advisera.com/13485academy/what-is-eu-mdr/#:~:text=Under%20the%20EU%20MDR%2C%20medical%20device%20manufacturers%20are,ensure%20ongoing%20compliance%20and%20safety%20of%20the%20devices'}, page_content='403 Forbidden')]

In [113]:
raw_text =urls[0].page_content
len(raw_text)

13

In [114]:
clean_url_text = clean(text=raw_text,
            fix_unicode=True,
            to_ascii=True,
            lower=True,
            no_line_breaks=False,
            no_urls=False,
            no_emails=False,
            no_phone_numbers=False,
            no_numbers=False,
            no_digits=False,
            no_currency_symbols=False,
            no_punct=False,
            lang="en"
            )

In [115]:
len(clean_url_text)

13

In [117]:
from langchain.schema import Document
metadata = {
    "source": 'https://finance.ec.europa.eu/financial-crime/anti-money-laundering-and-countering-financing-terrorism-eu-level_en#legislation'
}

doc = Document(page_content=clean_url_text, metadata=metadata)
document =[doc]

In [113]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
texts = text_splitter.split_documents(document)

In [114]:
len(texts)

60

In [115]:
start =time.time()
# Insert documents into the collection
qdrant = Qdrant.from_documents(texts,embeddings_model,url=url,prefer_grpc=False,api_key=qdrant_api_key,collection_name=collection_name,force_recreate=False,distance_func="Dot")
end =time.time()
print("Time taken is:", end-start,"seconds")

Time taken is: 6.629606485366821 seconds


In [116]:
query = "What is EU MDR compliance?"
qdrant.similarity_search_with_score(query)

[(Document(metadata={'source': 'https://advisera.com/13485academy/what-is-eu-mdr/#:~:text=Under%20the%20EU%20MDR%2C%20medical%20device%20manufacturers%20are,ensure%20ongoing%20compliance%20and%20safety%20of%20the%20devices.', '_id': '3e50f80c-3284-4696-8fcc-b0590e988a8a', '_collection_name': 'Healthcare_collection'}, page_content='Overall, the extension of the MDR provides a realistic timeline for all parties involved to adapt to the new regulatory landscape and to continue providing safe and high-quality medical devices to patients in the European Union.\n\nWhat is EU MDR compliance?'),
  0.8387462),
 (Document(metadata={'source': 'https://advisera.com/13485academy/what-is-eu-mdr/#:~:text=Under%20the%20EU%20MDR%2C%20medical%20device%20manufacturers%20are,ensure%20ongoing%20compliance%20and%20safety%20of%20the%20devices.', '_id': 'cd0308ca-23a2-4224-ab6d-64d420346b4d', '_collection_name': 'Healthcare_collection'}, page_content='Overall, EU MDR compliance plays a vital role in ensuring 